In [ ]:
# 将单个trajectory的action之中的一些字段提取出来，保存为csv文件，方便后续分析
import json, re
from pathlib import Path
import pandas as pd

def extract_actions_from_trace_json(path: Path):
    """
    Input:  WonderBread trace json file (contains {"trace":[...]}).
    Output: list[dict], each dict is a normalized action row.
    """
    obj = json.loads(path.read_text(encoding="utf-8"))
    trace = obj.get("trace", [])
    rows = []

    for idx, item in enumerate(trace):
        if item.get("type") != "action":
            continue

        a = item.get("data", {})
        el = (a.get("element_attributes") or {}).get("element") or {}

        # normalize element_text to avoid newline breaking CSV rows
        text = el.get("text")
        if isinstance(text, str):
            text_norm = text.replace("\r\n", "\n").replace("\r", "\n").replace("\n", "\\n").strip()
        else:
            text_norm = None

        rows.append({
            # optional metadata for debugging
            "trace_file": path.name,
            "trace_action_idx": idx,     # index in trace array

            # required: action fields
            "action_id": a.get("id"),
            "action_type": a.get("type"),

            # required: element fields
            "element_xpath": el.get("xpath"),
            "element_label": el.get("label"),
            "element_type": el.get("type"),
            "element_placeholder": el.get("placeholder"),
            "element_role": el.get("role"),
            "element_tag": el.get("tag"),
            "element_text": text_norm,
            "element_value": el.get("value"),
        })

    return rows

def extract_folder_to_csv(input_dir: str, output_csv: str):
    input_dir = Path(input_dir)

    # Find all JSON files in the directory
    json_files = sorted(input_dir.glob("*.json"))

    print(f"Found {len(json_files)} JSON files")
    for f in json_files:
        print(f"  - {f.name}")

    all_rows = []
    for p in json_files:
        all_rows.extend(extract_actions_from_trace_json(p))

    df = pd.DataFrame(all_rows)
    df.to_csv(output_csv, index=False, encoding="utf-8")
    print(f"Saved {len(df)} actions -> {output_csv}")

if __name__ == "__main__":
    extract_folder_to_csv(
        input_dir="./gold_demos_sop_and_trace_only/0 @ 2023-12-25-15-10-58", 
        output_csv="./result/0wonderbread_actions_normalized.csv"
    )


Found 1 JSON files
  - 0 @ 2023-12-25-15-10-58.json
Saved 9 actions -> ./result/0wonderbread_actions_normalized.csv


In [ ]:
# 将同一个task_id的多个trajectory的action提取出来，合并保存为一个csv文件，方便后续分析
from collections import defaultdict
import os

def extract_all_tasks_to_csv(base_dir: str, output_dir: str):
    """
    Process all folders in base_dir, group by task ID, and generate one CSV per task.
    
    Folder naming format: "{task_id} @ {timestamp}"
    Example: "0 @ 2023-12-25-15-10-58" -> task_id = 0
    """
    base_path = Path(base_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Group folders by task ID
    task_folders = defaultdict(list)
    
    # Scan all folders
    for folder in sorted(base_path.iterdir()):
        if not folder.is_dir():
            continue
        
        # Extract task ID from folder name (number before @)
        folder_name = folder.name
        match = re.match(r'^(\d+)\s*@', folder_name)
        if match:
            task_id = match.group(1)
            task_folders[task_id].append(folder)
    
    print(f"Found {len(task_folders)} unique tasks")
    print(f"Task IDs: {sorted(task_folders.keys(), key=lambda x: int(x))}\n")
    
    # Process each task
    for task_id in sorted(task_folders.keys(), key=lambda x: int(x)):
        folders = task_folders[task_id]
        print(f"Processing Task {task_id}: {len(folders)} trajectories")
        
        all_rows = []
        for folder in sorted(folders):
            # Find JSON files in this folder
            json_files = list(folder.glob("*.json"))
            print(f"  - {folder.name}: {len(json_files)} JSON files")
            
            for json_file in json_files:
                rows = extract_actions_from_trace_json(json_file)
                all_rows.extend(rows)
        
        # Save to CSV
        if all_rows:
            df = pd.DataFrame(all_rows)
            output_csv = output_path / f"{task_id}_traj.csv"
            df.to_csv(output_csv, index=False, encoding="utf-8")
            print(f"  ✓ Saved {len(df)} actions -> {output_csv.name}\n")
        else:
            print(f"  ⚠ No actions found for task {task_id}\n")
    
    print(f"\n{'='*60}")
    print(f"Processing complete!")
    print(f"Generated {len(task_folders)} CSV files in {output_dir}")
    print(f"{'='*60}")

# Run the batch processing
if __name__ == "__main__":
    extract_all_tasks_to_csv(
        base_dir="./gold_demos_sop_and_trace_only",
        output_dir="./result"
    )


Found 164 unique tasks
Task IDs: ['0', '1', '3', '4', '5', '14', '21', '29', '42', '44', '45', '47', '62', '63', '64', '69', '79', '94', '106', '107', '114', '115', '116', '117', '118', '122', '125', '127', '131', '132', '133', '143', '147', '156', '157', '162', '163', '171', '177', '179', '185', '188', '194', '203', '205', '212', '216', '226', '228', '235', '240', '245', '258', '259', '260', '276', '279', '280', '281', '282', '285', '296', '299', '317', '319', '322', '323', '327', '332', '333', '336', '343', '345', '350', '357', '360', '374', '384', '385', '386', '393', '402', '407', '410', '412', '413', '417', '422', '438', '440', '446', '450', '457', '461', '464', '468', '472', '477', '480', '483', '489', '499', '503', '508', '510', '512', '519', '521', '522', '524', '534', '539', '545', '547', '571', '578', '579', '580', '585', '593', '595', '601', '607', '613', '622', '627', '630', '637', '644', '645', '646', '647', '648', '649', '652', '654', '661', '669', '677', '693', '695', '7

In [11]:
# 分析单个task对应的多条trajectory的CSV中的高频action步骤，保存为新的CSV文件
from pathlib import Path
import pandas as pd
from IPython.display import display

def summarize_high_freq_actions(csv_path: str, min_count: int = 2, min_trace_coverage: int = 2, top_n: int = 40, output_dir: str = "frequency"):
    """
    Aggregate actions in one traj CSV to find high-frequency steps.
    Group by (action_type, element_text), compute frequency and typical position using the
    median of trace_action_idx, and sort results in the relative order they usually appear.
    """

    out_dir = Path(output_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    csv_path = Path(csv_path)
    df = pd.read_csv(csv_path)
    required_cols = {"trace_file", "trace_action_idx", "action_type", "element_text"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"CSV is missing required columns: {sorted(missing)}")

    df["element_text"] = df["element_text"].fillna("(missing)").astype(str)
    df["trace_action_idx"] = pd.to_numeric(df["trace_action_idx"], errors="coerce")
    df = df.dropna(subset=["trace_action_idx"])
    df["trace_action_idx"] = df["trace_action_idx"].astype(int)

    grouped = (
        df.groupby(["action_type", "element_text"], dropna=False)
        .agg(
            hits=("element_text", "size"),
            trace_files=("trace_file", pd.Series.nunique),
            median_order=("trace_action_idx", "median"),
            mean_order=("trace_action_idx", "mean"),
            min_order=("trace_action_idx", "min"),
            max_order=("trace_action_idx", "max"),
        )
        .reset_index()
    )

    filtered = grouped[
        (grouped["hits"] >= min_count) & (grouped["trace_files"] >= min_trace_coverage)
    ].copy()

    filtered["order_rank"] = (
        filtered["median_order"].rank(method="dense", ascending=True).astype(int)
    )
    filtered = filtered.sort_values(
        ["order_rank", "hits", "median_order"], ascending=[True, False, True]
    )

    if top_n:
        filtered = filtered.head(top_n)

    out_path = out_dir / f"{csv_path.stem}_highfreq.csv"
    filtered.to_csv(out_path, index=False, encoding="utf-8")
    print(f"Saved {len(filtered)} rows -> {out_path}")
    return filtered

# Example usage: analyze one trajectory CSV and display the ranked high-frequency actions
example_csv = "./result/0_traj.csv"
highfreq_df = summarize_high_freq_actions(
    example_csv,
    min_count=2,            # only keep actions seen at least twice
    min_trace_coverage=2,   # only keep actions appearing in at least 2 different trace_files
    top_n=30,               # trim to the top-N by order + frequency
)
display(highfreq_df)


Saved 7 rows -> frequency/0_traj_highfreq.csv


,action_type,element_text,hits,trace_files,median_order,mean_order,min_order,max_order,order_rank
10,mouseup,REPORTS,5,5,1.0,1.400000,1,3,1
6,mouseup,Bestsellers,5,5,3.0,3.400000,3,5,2
8,mouseup,Day\nMonth\nYear,8,4,6.0,6.000000,5,7,3
2,mouseup,(missing),8,4,10.0,10.500000,7,13,4
0,keystroke,(missing),7,4,13.0,13.000000,11,15,5
11,mouseup,Show Report,5,5,17.0,19.000000,15,29,6
13,scroll,(missing),3,3,19.0,18.333333,17,19,7
